In [ ]:
# pip install stop_words nltk pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 34.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=8d7e3397b8bcb90367568da72aee52d468f54b70291d39cfd00a608fb01d180d
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=d71d72100ac78eb31459eeb7d6be117464733b3738387c20c8be29d8fc451c6a
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built stop_words docopt


In [37]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

import keras
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.layers import SimpleRNN, LSTM, GRU, Masking, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from keras.callbacks import TensorBoard 
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy 
from keras.callbacks import EarlyStopping  

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 
from sklearn import model_selection, preprocessing, linear_model

from gensim.models import Word2Vec
from gensim import corpora
from collections import defaultdict, Counter

In [2]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\Alina\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

In [4]:
reviews = pd.read_excel('отзывы за лето.xls')

In [5]:
reviews.dropna(inplace=True)

In [6]:
morph = MorphAnalyzer()

def preprocess_text(txt):
  txt = str(txt)
  txt = txt.lower()
  txt = re.sub('[0-9]|[-—.,:;_%©«»?*!@#№$^•·"&()]|[+=]|[[]|[]]|[/]|[..]|[...]|[....]|[d]', ' ', txt)
  txt = [morph.parse(word)[0].normal_form for word in txt.split() if word not in stopwords.words('russian')]
  # txt = " ".join(txt)
  return txt

In [7]:
reviews['clean_text'] = reviews.Content.apply(preprocess_text)
reviews['label'] = reviews.Rating.map({5:0, 4:0, 3:1, 2:1, 1:1})

<ipython-input-6-9a365ce90dba>:6: FutureWarning: Possible nested set at position 40
  txt = re.sub('[0-9]|[-—.,:;_%©«»?*!@#№$^•·"&()]|[+=]|[[]|[]]|[/]|[..]|[...]|[....]|[d]', ' ', txt)


In [8]:
reviews.head(7)

,Rating,Content,Date,clean_text,label
0,5,It just works!,2017-08-14,"[it, just, works]",0
1,4,"В целом удобноное приложение...из минусов хотят слишком большой доступ к персональным данным в телефоне,приходится пользоваться в ограниченном режиме",2017-08-14,"[целое, удобноной, приложение, минус, хотеть, слишком, большой, доступ, персональный, данные, телефон, приходиться, пользоваться, ограниченный, режим]",0
2,5,Отлично все,2017-08-14,[отлично],0
3,5,Стал зависать на 1% работы антивируса. Дальше никуда. Ранее больше года пользовался нормально.,2017-08-14,"[стать, зависать, работа, антивирус, далёкий, никуда, ранее, год, пользоваться, нормально]",0
4,5,"Очень удобно, работает быстро.",2017-08-14,"[очень, удобно, работать, быстро]",0
5,5,Всё удобно норм 👍👍👍,2017-08-14,"[всё, удобно, норма, 👍👍👍]",0
6,5,Очень удобное приложение.,2017-08-14,"[очень, удобный, приложение]",0


In [9]:
train_df, valid_df = model_selection.train_test_split(reviews, random_state=678)

In [10]:
full_corpus = []

for text in train_df.clean_text:
  for word in text:
    full_corpus.append(word)

print(len(full_corpus))

87434


In [11]:
freq_dict = Counter(full_corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
len(freq_dict_sorted)

7354

In [12]:
high_freq_words = []
mid_freq_words = []
low_freq_words = []

for word, freq in freq_dict_sorted:
  if freq >= 100:
    high_freq_words.append(word)
  elif freq < 100 and freq >= 10:
    mid_freq_words.append(word)
  else: 
    low_freq_words.append(word)

len(high_freq_words), len(mid_freq_words), len(low_freq_words)

(137, 877, 6340)

In [13]:
vocab = high_freq_words + mid_freq_words
len(vocab)

1014

In [14]:
vec1 = CountVectorizer(lowercase=False, analyzer=lambda x: x, ngram_range=(1, 4), vocabulary=vocab)
x_train_cv = vec1.fit_transform(train_df.clean_text).toarray()
x_valid_cv = vec1.fit_transform(valid_df.clean_text).toarray()

In [15]:
vec2 = TfidfVectorizer(lowercase=False, analyzer=lambda x: x, ngram_range=(1, 4), vocabulary=vocab)
x_train_tfidf = vec2.fit_transform(train_df.clean_text).toarray()
x_valid_tfidf = vec2.fit_transform(valid_df.clean_text).toarray()

In [16]:
max_words = len(freq_dict_sorted)
max_len = len(vocab)

# Training
epochs = 20
batch_size = 128
print_batch_n = 100

In [17]:
x_train_cv.shape, x_train_tfidf.shape 

((15492, 1014), (15492, 1014))

In [18]:
x_valid_cv.shape, x_valid_tfidf.shape

((5164, 1014), (5164, 1014))

In [19]:
num_classes = 2
y_train = keras.utils.to_categorical(train_df.label, num_classes)
y_valid = keras.utils.to_categorical(valid_df.label, num_classes)

In [20]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [21]:
model.compile(loss='CategoricalCrossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [22]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train_cv, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])


Epoch 1/20
109/109 [==============================] - 22s 117ms/step - loss: 0.4635 - accuracy: 0.8183 - val_loss: 0.4358 - val_accuracy: 0.8310
Epoch 2/20
109/109 [==============================] - 12s 112ms/step - loss: 0.4363 - accuracy: 0.8281 - val_loss: 0.4381 - val_accuracy: 0.8323


In [23]:
score = model.evaluate(x_valid_cv, y_valid, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

41/41 [==============================] - 2s 48ms/step - loss: 0.4547 - accuracy: 0.8292


Test score: 0.45471957325935364
Test accuracy: 0.8292021751403809


In [24]:
y_train_rnn = train_df.label.values
y_valid_rnn = valid_df.label.values

In [25]:
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=max_words, output_dim=30, input_length=max_len, trainable=True, mask_zero=True))
model_lstm.add(Masking(mask_value=0.0))
model_lstm.add(LSTM(64, recurrent_dropout=0.2))
model_lstm.add(Dense(64, activation='relu'))
model_lstm.add(Dropout(0.5))
model_lstm.add(Dense(1, activation='sigmoid'))

In [26]:
model_lstm.compile(loss='BinaryCrossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [27]:
early_stopping=EarlyStopping(monitor='val_loss')  

history = model_lstm.fit(x_train_cv, y_train_rnn,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])


Epoch 1/20
109/109 [==============================] - 830s 8s/step - loss: 0.5073 - accuracy: 0.8116 - val_loss: 0.4822 - val_accuracy: 0.8142
Epoch 2/20
109/109 [==============================] - 789s 7s/step - loss: 0.4911 - accuracy: 0.8127 - val_loss: 0.4820 - val_accuracy: 0.8142


In [28]:
score = model_lstm.evaluate(x_valid_cv, y_valid_rnn, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

41/41 [==============================] - 28s 676ms/step - loss: 0.5067 - accuracy: 0.7994


Test score: 0.506669819355011
Test accuracy: 0.7993803024291992


In [29]:
model_gru = Sequential()
model_gru.add(Embedding(input_dim=max_words, output_dim=30, input_length=max_len, trainable=True, mask_zero=True))
model_gru.add(Masking(mask_value=0.0))
model_gru.add(GRU(64, recurrent_dropout=0.2))
model_gru.add(Dense(64, activation='relu'))
model_gru.add(Dropout(0.5))
model_gru.add(Dense(1, activation='sigmoid'))

In [30]:
model_gru.compile(loss='BinaryCrossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [31]:
early_stopping=EarlyStopping(monitor='val_loss')  

history = model_gru.fit(x_train_cv, y_train_rnn,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])


Epoch 1/20
109/109 [==============================] - 730s 7s/step - loss: 0.5113 - accuracy: 0.8120 - val_loss: 0.4800 - val_accuracy: 0.8142
Epoch 2/20
109/109 [==============================] - 715s 7s/step - loss: 0.4904 - accuracy: 0.8127 - val_loss: 0.4816 - val_accuracy: 0.8142


In [32]:
score = model_gru.evaluate(x_valid_cv, y_valid_rnn, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

41/41 [==============================] - 26s 633ms/step - loss: 0.5014 - accuracy: 0.7994


Test score: 0.5014299750328064
Test accuracy: 0.7993803024291992


In [44]:
model_comb = Sequential()
model_comb.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model_comb.add(Conv1D(128, 3))
model_comb.add(Activation("relu"))
model_comb.add(MaxPooling1D(pool_size=2))
model_comb.add(LSTM(100))
# model_comb.add(Dropout(0.2))
model_comb.add(Dense(1, activation='sigmoid'))

In [45]:
model_comb.compile(loss='BinaryCrossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [46]:
early_stopping=EarlyStopping(monitor='val_loss')  

history = model_comb.fit(x_train_cv, y_train_rnn,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
109/109 [==============================] - 25s 206ms/step - loss: 0.4935 - accuracy: 0.8063 - val_loss: 0.4818 - val_accuracy: 0.8142
Epoch 2/20
109/109 [==============================] - 22s 201ms/step - loss: 0.4823 - accuracy: 0.8127 - val_loss: 0.4760 - val_accuracy: 0.8155


In [47]:
score = model_comb.evaluate(x_valid_cv, y_valid_rnn, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

41/41 [==============================] - 3s 80ms/step - loss: 0.4985 - accuracy: 0.7978


Test score: 0.4984525740146637
Test accuracy: 0.797831118106842


In [60]:
model_comb2 = Sequential()
model_comb2.add(Embedding(input_dim=max_words, output_dim=30, input_length=max_len, trainable=True, mask_zero=True))
model_comb2.add(LSTM(128, return_sequences=True))
model_comb2.add(Conv1D(128, 3))
model_comb2.add(Activation("relu"))
model_comb2.add(MaxPooling1D(pool_size=2))
model_comb2.add(Conv1D(64, 3))
model_comb2.add(Activation("relu"))
model_comb2.add(GlobalMaxPool1D())
# model_comb.add(Dropout(0.2))
model_comb2.add(Dense(1, activation='sigmoid'))

In [61]:
model_comb2.compile(loss='BinaryCrossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [62]:
early_stopping=EarlyStopping(monitor='val_loss')  

history = model_comb2.fit(x_train_cv, y_train_rnn,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
109/109 [==============================] - 455s 4s/step - loss: 0.4324 - accuracy: 0.8261 - val_loss: 0.3857 - val_accuracy: 0.8381
Epoch 2/20
109/109 [==============================] - 454s 4s/step - loss: 0.3631 - accuracy: 0.8411 - val_loss: 0.3774 - val_accuracy: 0.8400


In [64]:
score = model_comb2.evaluate(x_valid_cv, y_valid_rnn, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

41/41 [==============================] - 21s 515ms/step - loss: 0.3694 - accuracy: 0.8490


Test score: 0.36941367387771606
Test accuracy: 0.8489543199539185


In [65]:
model_comb3 = Sequential()
model_comb3.add(Embedding(input_dim=max_words, output_dim=30, input_length=max_len, trainable=True, mask_zero=True))
model_comb3.add(LSTM(128, return_sequences=True))
model_comb3.add(Conv1D(128, 3))
model_comb3.add(Activation("relu"))
model_comb3.add(GlobalMaxPool1D())
# model_comb.add(Dropout(0.2))
model_comb3.add(Dense(1, activation='sigmoid'))

In [66]:
model_comb3.compile(loss='BinaryCrossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [67]:
early_stopping=EarlyStopping(monitor='val_loss')  

history = model_comb3.fit(x_train_cv, y_train_rnn,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
109/109 [==============================] - 447s 4s/step - loss: 0.4460 - accuracy: 0.8282 - val_loss: 0.4094 - val_accuracy: 0.8290
Epoch 2/20
109/109 [==============================] - 439s 4s/step - loss: 0.3867 - accuracy: 0.8389 - val_loss: 0.3957 - val_accuracy: 0.8348


In [68]:
score = model_comb3.evaluate(x_valid_cv, y_valid_rnn, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

41/41 [==============================] - 20s 482ms/step - loss: 0.3841 - accuracy: 0.8373


Test score: 0.38406020402908325
Test accuracy: 0.8373354077339172


Выводы:
Рекуррентные сети в целом показали худшие результаты, чем сверточные. Комбинация CNN->LSTM закономерно обучается гораздо быстрее, чем LSTM/GRU сами по себе, но результат показывает схожий. Самый лучший результат показывает комбинация LSTM->CNN.